<a href="https://colab.research.google.com/github/evainnocente/537-project/blob/main/pcadapt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pcadapt

creating bed files to analyse with pcadapt- one for non survivors and one for survivors and immune responders

to get the vcf for each group: merge all the individual sample vcfs from each group

In [ ]:
plink --allow-extra-chr --double-id --vcf NSvariants.vcf --make-bed --out NSvariants

Pcadapt (following this tutorial in R:https://bcm-uga.github.io/pcadapt/articles/pcadapt.html)

In [ ]:
```{r}
library(pcadapt)

path_to_file <- "C:/Users/evain/OneDrive - Queen's University/university/537/Population diversity analyses/NSvariants.bed"

data<-read.pcadapt(path_to_file, type = "bed")
x <- pcadapt(input = data, K=4)

#screeplot says first 2 PCs are best-retain pcs to left of straight line according to Cattell's rule
plot(x, option = "screeplot")

#making x have optimal number of PCS (2)- rename to y
y<- pcadapt(input=data, K = 2)
summary(y)
y$pvalues

#score plot with first 2 PCs
plot(y, option = "scores")

#with PC 3 and 4- not as good
plot(x, i = 3, j = 4, option = "scores")

#manhattan plot
plot(y , option = "manhattan")

#qq plot- try with x and y
plot(y, option = "qqplot")

#if (!require("BiocManager", quietly = TRUE))
    #install.packages("BiocManager")

BiocManager::install("qvalue")

library(qvalue)
qval <- qvalue(y$pvalues)$qvalues
alpha <- 0.05 #in the tutorial they use 0.1- what value of alpha is best?
outliers <- which(qval < alpha)
length(NSoutliers)
NSoutliers<-outliers

#try with bonferroni correction- fewer outliers
padj <- p.adjust(y$pvalues,method="bonferroni")
alpha <- 0.05
outliers <- which(padj < alpha)
length(outliers)

length(qval)

print(NSoutliers)

NSoutliers<-as.data.frame(NSoutliers)
NSoutliers
write.csv(NSoutliers,file="NSoutliers.csv")
outliers

#repeat for survivors and immune responders
```

Narrowing down the list of outliers to TLR outliers only

Basically the list of outliers created by pcadapt is a list of the lines in the bim file that contain significant outliers- e.g. if the pcadapt outliers say 2, 4, 9 then lines 2, 4, and 9 of the bim file have the outlying snp- so I just matched them but of course there are outliers outside of TLRs, so I first removed chromosomes that don't have TLRs. Then I looked through the lists manually and cross referenced them with the TLR positions I got from the common eider annotation, and kept only the outliers that fell within TLRs.

TLR positions:

tlr1a SUPER_4:53,881,681-53,884,137

tlr1b SUPER_4:53,873,395-53,875,356

TLR2A: SUPER_4:32,243,726-32,250,107
TLR2B: SUPER_4:32,234,631-32,240,982

TLR3: SUPER_4:20,097,425-20,107,088

tlr4 SUPER_18:8,037,993-8,042,694

tlr5 SUPER_3:95,457,095-95,459,677

TLR7: SUPER_1:77,565,800-77,578,017

TLR15: SUPER_3:116,325,458-116,332,055

TLR21: SUPER_29:1,577,922-1,588,503

In [ ]:
tr '\r' '\n' < SIRoutliers.csv > SIRoutliers_unix.csv # corrects line ending characters or something like that so that we can search through it
awk 'FNR==NR{line[$1]; next} FNR in line' SIRoutliers_unix.csv SIRvariants.bim > SIRoutliervariants.txt #the bim file was created when we made the bed file and is a list of all snps
sed '/^SUPER_2/ d' <  SIRoutliervariantsTLRsonly.txt > no2SIR.txt #removed all irrelevant chromosomes with this

You can remove multiple chromosomes at a time by using \ | as a delimiter

In [ ]:
sed '/^SUPER_7\|^SUPER_8\|^SUPER_9\|^SUPER_10\|^SUPER_11\|^SUPER_12\|^SUPER_13\|^SUPER_14\|^SUPER_15\|^SUPER_16\|^SUPER_17\|^SUPER_19\|^SUPER_20\|^SUPER_21\|^SUPER_22\|^SUPER_23\|^SUPER_24\|^SUPER_25\|^SUPER_26\|^SUPER_27\|^SUPER_28\|^SUPER_30\|^SUPER_31\|^SUPER_32\|^SUPER_Z\|^SUPER_W/d'